<a href="https://colab.research.google.com/github/NUG30/homework-2-Guozhang2020/blob/main/Assignment2_LinGuozhang061801907.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Homework 2: Naive Bayes Email Spam Filter

In this homework, we will learn how to implement the Naive Bayes classifier in order to create a simple email spam filter. This spam filter will be trained by test_emails, which will be given by a vector of tuples (emails, spam/nospam). For each tuple the first entry is a string ("email"), and the second entry is 0 or 1, depending whether the email contains spam words.

In [4]:
dictionary = ['hello', 'students', 'please', 'learn', 'for', 'the', 'exam', 'buy', 'drugs', 'today', 'sun', 'is', 'shining', 'in', 'nagoya', 'lets', 'sell', 'how', 'are', 'you', 'today?', 'do', 'your', 'homework', 'want', 'free', 'solutions?', 'hey', 'always', 'ask', 'questions', 'if','have', 'any', 'math', 'not', 'good', 'submit', 'pay'] 

test_emails=[
             ["hello students, please learn for the exam", 0],
             ["hello students, please buy drugs", 1],    
             ["hello, today the sun is shining in nagoya", 0],
             ["lets sell drugs in nagoya", 1],
             ["today learn drugs", 1],
             ["how are you today?", 0],
             ["hello students, please do your homework", 0],
             ["hello, do you want free homework solutions?", 1],
             ["hey students, please always ask questions if you have any", 0],
             ["math is not good", 1],
             ["math is good", 0],
             ["please submit your homework", 0],
             ["please buy questions", 1],
             ["please pay for the exam", 1]          
             ]

The feature space for our spam filter will be $\mathcal{X}=\{0,1\}^d$, where $d$ denotes the number of words in the dictionary. For a feature (email) $x \in \mathcal{X}$ the entry $x_i$ for $i=1,\dots,d$ is $1$ if the $i$-th word of the dictionary is contained in the email and $0$ otherwise.  

# **Exercise 1**
Implement a function which creates a feature vector out of an email and a function which creates a training set out of test emails. 

You would need to figure out whether a sentence contains a word, and there are functions in Python that could determine whether a string contains another string. You can consult documentation (and Google) to find out.

In [5]:
import numpy as np
def email_to_feature(dict, email):
  d = len(dict)      #d denotes # of words in the dictionary

#Construct an element in feature space corresponding to the email
  vector = np.zeros(d) #Vector in feature space. Default is all zeros
  for i in range(d):
    if dict[i] in email: #When the word is in the email,
      vector[i]=1     #the corresponding entry changes to 1
  return vector

def emails_to_test_samples(dict, test_emails):
  test_samples=test_emails.copy()
  n = len(test_samples)  #n denotes # of training samples

#Construct features according to emails in test_emails and put them in test_samples
  for i in range(n):
    test_samples[i][0]=email_to_feature(dict,test_emails[i][0])
  return test_samples
#Now for test_samples[j][k][i],
#j means j'th sample
#if k is 0, then it is an element in feature space
#and i means it is related to i'th word in dictionary
#if k is 1, then it is an element in label space


 **Recall from Lecture 6:**

Given a training set  $\mathcal{T} = \left( (x^{(1)}, y^{(1)}) , \dots, (x^{(n)}, y^{(n)})   \right)$ we calculate for $i=1,\dots,d$ the following numbers
\begin{align*}
\phi_{i\mid y=1} &= \frac{1+\sum_{j=1}^n I(x^{(j)}_i = 1  \,\wedge \, y^{(j)}=1 ) }{2+\sum_{j=1}^n I(y^{(j)}=1)}\,,\\
	\phi_{i\mid y=0} &= \frac{1+\sum_{j=1}^n I(x^{(j)}_i = 1  \,\wedge \, y^{(j)}=0 )}{2+\sum_{j=1}^n I(y^{(j)}=0)}\,,\\
		\phi_{y=1} &= \frac{1+\sum_{j=1}^n I(y^{(j)} = 1)}{2+n} \,.
\end{align*}
Here $I$ denoted the indicator function. We used the laplace smoothing (thats why we have the $1+$ and $2+$) in order to make sure that we will not assume probability $0$ for unknown words.

Now assume we get a new feature (i.e. someone sends us an email) $x \in \{0,1\}^d$. Then we can calculate for each word $i=1,\dots,d$ the probabilities
\begin{align*}
P(x_i = 1 \mid y=1) &= \phi_{i\mid y=1}\,,\qquad &&P(x_i = 1 \mid y=0) = \phi_{i\mid y=0} \,,\\
P(x_i = 0 \mid y=1) &= 1- \phi_{i\mid y=1}\,,\qquad &&P(x_i = 0 \mid y=0) = 1-\phi_{i\mid y=0} \,. \\
\end{align*}

By the Naive Bayes assumption we have for $x \in \{0,1\}^d$
\begin{align*}
P(x \mid y)  = \prod_{i=1}^d P(x_j \mid y)\,.
\end{align*}
The probability of the new email being spam is then
\begin{align*}
P(y=1 \mid x) &= \frac{ P(x \mid y=1) P(y=1)}{P(x)}\\
&= \frac{\prod_{i=1}^d P(x_i \mid y = 1) \cdot \phi_{y=1}}{\prod_{i=1}^d P(x_i \mid y = 1) \cdot \phi_{y=1} + \prod_{i=1}^d P(x_i \mid y = 0) (1-\phi_{y=1})}\,.
\end{align*}


# **Exercise 2:** 
Use the above explanation of the Naive Bayes Spam filter and implement a function which gives the probability of an email being spam given the trainings email above. 


Test your spam filter with the following email.


In [6]:
test_samples=emails_to_test_samples(dictionary, test_emails)
d=len(dictionary)
n=len(test_emails)
#Constructing the variable for phi_i|y=1
phi_i_y1=np.zeros(d)
for i in range(d):
  sum1=0
  sum2=0
  for j in range(n):
    if test_samples[j][0][i]==1 and test_samples[j][1]==1:
      sum1=sum1+1
    if test_samples[j][1]==1:
      sum2=sum2+1
  phi_i_y1[i]=(1+sum1)/(2+sum2)

#Constructing the variable for phi_y=1
phi_y1=(1+sum2)/(2+n)

#Constructing the variable for phi_i|y=0
phi_i_y0=np.zeros(d)
for i in range(d):
  sum1=0
  sum2=0
  for j in range(n):
    if test_samples[j][0][i]==1 and test_samples[j][1]==0:
      sum1=sum1+1
    if test_samples[j][1]==0:
      sum2=sum2+1
  phi_i_y0[i]=(1+sum1)/(2+sum2)

def spam_percentage(dict,email):
#Convert email to element in feature space
  vector = email_to_feature(dict,email)

#Calculation based on the formulas
  product1=1
  product2=1
  for i in range(d):
    term1=1
    term2=1
    if vector[i]==1:
      term1=phi_i_y1[i]
      term2=phi_i_y0[i]
    if vector[i]==0:
      term1=1-phi_i_y1[i]
      term2=1-phi_i_y0[i]
    product1=term1*product1
    product2=term2*product2
  P=(product1*phi_y1/(product1*phi_y1+product2*(1-phi_y1)))
  return P


In [7]:
email="the sun is shining. buy drugs now"
print(spam_percentage(dictionary,email))

0.9299429164504411



# **Exercise 3**
Extend your spamfilter by creating a dynamical dictionary. Instead of starting with a fixed dictionary, you should now create a dictionary out of a list of emails. 

Write a function `create_dictionary(emails)` which returns a dictionary created from a list of emails (Give as an array of arrays `[text, spam\nospam]`). Make sure that you do not include words more than once into the dictionary.
To implement this function you should look up the function `split()` for a string in Python. To take care of the symbols "." and "," you can use the `replace()` function of a string.

In [11]:
import string
def create_dictionary(emails):
  n = len(emails)
  emails_without_y_and_punctations=[]
  split_emails=[]
  for i in range(n):
    #Use translate method to remove punctuations
    emails_without_y_and_punctations.append(str(emails[i][0]).translate(str.maketrans('','',string.punctuation)))
    split_emails.append(emails_without_y_and_punctations[i].split())
  dictionary=[]
  for i in range(n): #iterate over emails
    for word in split_emails[i]: #iterate over words in each email
      if word not in dictionary:
        dictionary.append(word)

  return(dictionary)
